In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
import numpy as np
private_labels=np.load("Nohot_private_label.npy")
private_pixels=np.load("private_pixels.npy")
public_labels=np.load("Nohot_public_label.npy")
public_pixels=np.load("public_pixels.npy")
train_labels=np.load("Nohot_train_label.npy")
train_pixels=np.load("train_pixels.npy")

In [3]:
X_train , y_train = train_pixels,train_labels
X_valid , y_valid = public_pixels,public_labels
X_test , y_test = private_pixels,private_labels

In [4]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals)/std_val
X_valid_centered = (X_valid - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val

In [5]:
import torch
import torch.nn
input_data=torch.FloatTensor(X_train_centered)
label=torch.LongTensor(y_train)

In [6]:
input_data1=torch.FloatTensor(X_valid_centered)
label1=torch.LongTensor(y_valid)

In [7]:
input_data2=torch.FloatTensor(X_test_centered)
label2=torch.LongTensor(y_test)

In [8]:
input_data=input_data.transpose(1,3)
input_data1=input_data1.transpose(1,3)
input_data2=input_data2.transpose(1,3)

In [9]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
trainset = TensorDataset(input_data,label)
valset = TensorDataset(input_data1,label1)
testset = TensorDataset(input_data2,label2)

In [10]:
partition={'train':trainset,'val':valset,'test':testset}

In [11]:
import torch
import torch.nn as nn

In [12]:
class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.residual_function = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, stride=stride, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, bias=False),
        nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )
        
        self.shortcut = nn.Sequential()
        
        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BottleNeck.expansion, stride=stride, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels * BottleNeck.expansion))
    def forward(self, x):
        return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))
            

In [13]:
class ResNet(nn.Module):
    
    def __init__(self, block, num_block, num_classes=7):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True))
        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        output = self.conv3_x(output)
        output = self.conv4_x(output)
        output = self.conv5_x(output)
        output = self.avg_pool(output)
        output = output.view(output.size(0), -1)
        output = self.fc(output) # => 이미지 크기가 
                                 #바뀌면 이 줄 코드를 # 하고 dimension 확인

        return output

In [14]:
def resnet152():
    return ResNet(BottleNeck, [3, 8, 36, 3])

In [15]:
def  ResNet50() :
     return ResNet(BottleNeck,[ 3 , 4 , 6 , 3 ])

In [16]:
def dimension_check():
    net=ResNet50()
    x= torch.randn(1,1,48,48) # 28 이미지
    y=net(x)
    print(y.size())

In [17]:
dimension_check()

torch.Size([1, 7])


In [18]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=128,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [19]:
def test(net, partition):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=128, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [20]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=128, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [21]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [22]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net=ResNet50()
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(100):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
test_acc = test(net,partition)
test_acc=round(test_acc,2)

  0%|          | 0/225 [00:00<?, ?it/s]

time : 83.55, train_loss : 1.84, train_acc : 27.13,val_acc : 27.86,val_loss : 1.74


  0%|          | 0/225 [00:00<?, ?it/s]

time : 84.84, train_loss : 1.58, train_acc : 37.87,val_acc : 41.85,val_loss : 1.64


  0%|          | 0/225 [00:00<?, ?it/s]

...

  0%|          | 0/225 [00:00<?, ?it/s]

time : 84.55, train_loss : 0.06, train_acc : 97.91,val_acc : 55.17,val_loss : 3.22


  0%|          | 0/29 [00:00<?, ?it/s]

time : 84.68, train_loss : 0.09, train_acc : 96.84,val_acc : 56.59,val_loss : 2.80


100%|██████████| 29/29 [00:02<00:00, 10.02it/s]


In [23]:
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/resnet_train_loss.npy",train_losses)
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/resnet_val_loss.npy",val_losses)
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/resnet_train_acc.npy",train_accs)
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/resnet_val_acc.npy",val_accs)